In [ ]:
%pip install -r requirements.txt

In [19]:
from transformers import CLIPModel, CLIPTokenizer, CLIPImageProcessor, AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import torch
import torch.nn as nn

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
vision_encoder = clip_model.vision_model
text_encoder = clip_model.text_model
image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B-Base")
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B-Base")


In [17]:
items = ["cat", "dog", "horse", "bird", "car", "person", "tree", "house", "book", "phone"]
classifiers = [f"a photo of a {item}" for item in items]

text_inputs = tokenizer(classifiers, return_tensors="pt")
text_features = text_encoder(**text_inputs).pooler_output
text_features_proj = clip_model.text_projection(text_features)

for item, classifier in zip(items, classifiers):
    image = Image.open(f"images/{item}.jpg")
    image_inputs = image_processor(image, return_tensors="pt")
    image_features = vision_encoder(**image_inputs).pooler_output
    image_features_proj = clip_model.visual_projection(image_features)
    
    # Compute similarity to all classifier texts
    similarities = (image_features_proj @ text_features_proj.T).squeeze(0)
    best_idx = similarities.argmax().item()
    best_label = classifiers[best_idx]
    best_score = similarities[best_idx].item()
    
    print(f"{item}: best match is '{best_label}' (score: {best_score:.3f})")

cat: best match is 'a photo of a cat' (score: 30.380)
dog: best match is 'a photo of a dog' (score: 35.998)
horse: best match is 'a photo of a horse' (score: 36.964)
bird: best match is 'a photo of a bird' (score: 30.397)
car: best match is 'a photo of a car' (score: 23.982)
person: best match is 'a photo of a person' (score: 27.901)
tree: best match is 'a photo of a tree' (score: 30.656)
house: best match is 'a photo of a house' (score: 29.412)
book: best match is 'a photo of a book' (score: 40.067)
phone: best match is 'a photo of a phone' (score: 32.127)


In [ ]:
clip_dim = clip_model.visual_projection.out_features  # 512
qwen_dim = qwen_model.model.embed_tokens.embedding_dim  # 4096

adapter = nn.Sequential(
    nn.Linear(clip_dim, qwen_dim),
    nn.LayerNorm(qwen_dim),
    nn.GELU(),
    nn.Linear(qwen_dim, qwen_dim),
)

for item in items:
    image = Image.open(f"images/{item}.jpg")
    image_inputs = image_processor(image, return_tensors="pt")
    image_features = vision_encoder(**image_inputs).pooler_output
    image_features_proj = clip_model.visual_projection(image_features)
    image_latent = adapter(image_features_proj)
    attention_mask = torch.ones(1, 1)
    position_ids = torch.zeros(1, 1)

    generated_ids = qwen_model.generate(inputs_embeds=image_latent.unsqueeze(1), attention_mask=attention_mask, position_ids=position_ids, max_new_tokens=30, do_sample=True, temperature=0.8, pad_token_id=qwen_tokenizer.pad_token_id, eos_token_id=qwen_tokenizer.eos_token_id)

    print(f"{item}: {qwen_tokenizer.decode(generated_ids[0])}")

cat: 乐观 Charter Candle Rescue作文住景供想象 Kiss charter试验今天的景 charter赎回收景acc1追赶景收景景景景景收募
dog: ii️À########
^)|;;ii️NN}}}️ennyII^^II|enny;;\tBBBB}}}iii×????ennyaaaaaaaa|
BBBB
horse: 磨 implied追赶.Unicode.Parcel.Parcelchargesaces派par.Parcel cualquier苗派 Paramountaces Alma Alma派 nar narкультур   charges nar captures派 nar追赶追赶
bird: 7完整 E base¡ |完全NC.subtract Letters住完整的 favorable |完整 Waveadera完全 Charterеспدد pledge Rدد¡دد7دد LIVENC
car: 绣_int绣夏夏rom只绣 rom绣为主绣owniorown绣夏绣ottie彩绣_rgb_rom高清高清为主rom只loat_pr
person: bohydrbohydrbohydrbohydrbohydrbohydrbohydrbohydrbohydrientbohydr่ายbohydrbohydrbohydrbohydrbohydr heatbohydrbohydrbohydrbohydrbohydrbohydrbohydrbohydrbohydr่ายbohydrbohydr
tree: uuoo^^¨oooaaaaÿÄeeeeoooooaaÄ^^??mm❤eeeeXXeeehhoooaaaaabcdooxxxxxxuuaaaa||
house: ện Labourар قناufficient قناufficientacious Labourufficient Labour...
iverse lợiiverseiverseufficient่ายufficientufficientufficientufficientacious labourLabour Labour Labourufficient laborufficient
book: 陪汗水微陪ート陪 armour陪�ooled armour汗水微耐�兼

In [32]:
# Download Flickr8k and fine-tune CLIP+Qwen (minimal demo)

import os
import requests
import zipfile
from PIL import Image
from tqdm import tqdm

# Download Flickr8k images
if not os.path.exists("Flickr8k_Dataset"):
    print("Downloading Flickr8k images...")
    url = "https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip"
    r = requests.get(url, stream=True)
    with open("Flickr8k_Dataset.zip", "wb") as f:
        for chunk in tqdm(r.iter_content(chunk_size=8192)):
            f.write(chunk)
    with zipfile.ZipFile("Flickr8k_Dataset.zip", "r") as zip_ref:
        zip_ref.extractall(".")

# Download Flickr8k captions
if not os.path.exists("Flickr8k_text"):
    print("Downloading Flickr8k captions...")
    url = "https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip"
    r = requests.get(url, stream=True)
    with open("Flickr8k_text.zip", "wb") as f:
        for chunk in tqdm(r.iter_content(chunk_size=8192)):
            f.write(chunk)
    with zipfile.ZipFile("Flickr8k_text.zip", "r") as zip_ref:
        zip_ref.extractall(".")

# Parse captions
captions = {}
with open("Flickr8k.token.txt", "r") as f:
    for line in f:
        img, caption = line.strip().split('\t')
        img = img.split('#')[0]
        captions.setdefault(img, []).append(caption)

# Minimal training loop (single image-caption pair per step, for demo)
from transformers import CLIPModel, CLIPImageProcessor, AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn as nn
import torch.optim as optim

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
vision_encoder = clip_model.vision_model
image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B-Base", torch_dtype=torch.float16)
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B-Base")

projection = nn.Linear(clip_model.visual_projection.out_features, qwen_model.model.embed_tokens.embedding_dim, dtype=torch.float16)
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = clip_model.to(device)
vision_encoder = vision_encoder.to(device)
qwen_model = qwen_model.to(device)
projection = projection.to(device)

optimizer = optim.AdamW(list(projection.parameters()) + list(qwen_model.parameters()), lr=1e-5)

# Training loop (VERY minimal, just for demonstration)
image_files = list(captions.keys())  # Use all images for demo
for epoch in range(10):  # Increase for real training
    for img_file in tqdm(image_files):
        img_path = os.path.join("Flickr8k_Dataset", "Flicker8k_Dataset", img_file)
        if not os.path.exists(img_path): continue
        caption = captions[img_file][0]  # Use the first caption

        # Image to latent
        image = Image.open(img_path).convert("RGB")
        image_inputs = image_processor(image, return_tensors="pt").to(device)
        image_features = vision_encoder(**image_inputs).pooler_output
        image_features_proj = clip_model.visual_projection(image_features)
        image_latent = projection(image_features_proj.half())

        # Caption to tokens
        input_ids = qwen_tokenizer(caption, return_tensors="pt", truncation=True, max_length=32).input_ids.to(device)
        text_embeds = qwen_model.model.embed_tokens(input_ids)
        decoder_input = torch.cat([image_latent.unsqueeze(1), text_embeds], dim=1)

        # Shift labels for causal LM
        labels = input_ids.clone()
        outputs = qwen_model(inputs_embeds=decoder_input, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item():.4f}")

print("Training loop complete (demo).")

image_folder = "images"
image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]

for img_file in image_files:
    image = Image.open(os.path.join(image_folder, img_file)).convert("RGB")
    image_inputs = image_processor(image, return_tensors="pt").to(device)
    image_features = vision_encoder(**image_inputs).pooler_output
    image_features_proj = clip_model.visual_projection(image_features)
    image_latent = projection(image_features_proj.half())

    # Generate text from image latent
    attention_mask = torch.ones(1, 1, dtype=torch.long).to(device)
    position_ids = torch.zeros(1, 1, dtype=torch.long).to(device)
    generated_ids = qwen_model.generate(
        inputs_embeds=image_latent.unsqueeze(1),
        attention_mask=attention_mask,
        position_ids=position_ids,
        max_new_tokens=30,
        do_sample=True,
        temperature=0.8,
        pad_token_id=qwen_tokenizer.pad_token_id,
        eos_token_id=qwen_tokenizer.eos_token_id,
    )
    description = qwen_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(f"{img_file}: {description}")

136160it [00:09, 14316.11it/s]


286it [00:00, 2124.61it/s]
100%|██████████| 8092/8092 [00:00<00:00, 525074.77it/s]


Training loop complete (demo).
dog.jpg: uuhanwwhanvvvvnnhanuu----------------------------------------------------------------------uuii:)uuiiuu~-)||ennyarry--------------------------------------------------------------------------------------------------------------------------------------------^^napjjoooo----------------------------------------------------------------------ennyiiuu
horse.jpg: aaaaaaaa''','''������������>>>>>>>>^^^^^^;;����cccc^^ooooaaaa||,,aaaaaaaauu||||^^ffffff;;&&>>>>>>>>^^','','                                                ","
book.jpg:  أما أما أما“ أما أما أما乔 أما أما أماечно أما أما أما أما أما أما أما أما恒 أما أما恒 أما أما أما外地 sağlam أما
bird.jpg: |||| jj;j;j------------->>>>>>>>;p&&||||;;;;jj XXX^^;;||||+
^^;;+
}}}||||]|}}}iiaaaaaaaa]|;p;;;;;;;;;;;;;;;;aaaa||||
person.jpg: ъъъappendъъъъъъъprint________________________________ъъъ:)ъъъъъъъъъъъъъ
house.jpg: aghdehyde(companyagh写作没写下aghputeraghaghubernetesgreSQLgreSQL耐磨agh写ubernetesWAREorry unix(...)
没 unix]-

In [ ]:
from tqdm import tqdm
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from transformers import CLIPModel, CLIPImageProcessor, AutoModelForCausalLM, AutoTokenizer

# Prepare captions dict
captions = {}
with open("Flickr8k.token.txt", "r") as f:
    for line in f:
        img, caption = line.strip().split('\t')
        img = img.split('#')[0]
        captions.setdefault(img, []).append(caption)

# Prepare list of (img_path, caption)
image_dir = "Flicker8k_Dataset"
pairs = []
for img in captions:
    img_path = os.path.join(image_dir, img)
    if os.path.exists(img_path):
        pairs.append((img_path, captions[img][0]))

# Model setup
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
vision_encoder = clip_model.vision_model
image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B-Base", torch_dtype=torch.float16)
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B-Base")

# Use float32 for projection for stability
projection = nn.Linear(clip_model.visual_projection.out_features, qwen_model.model.embed_tokens.embedding_dim, dtype=torch.float32)
nn.init.xavier_uniform_(projection.weight)
if projection.bias is not None:
    nn.init.zeros_(projection.bias)

# Device selection
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
clip_model = clip_model.to(device)
vision_encoder = vision_encoder.to(device)
qwen_model = qwen_model.to(device)
projection = projection.to(device)

# Freeze Qwen, only train projection
for param in qwen_model.parameters():
    param.requires_grad = False
for param in projection.parameters():
    param.requires_grad = True

optimizer = optim.AdamW(projection.parameters(), lr=1e-7)

# Collate function for batching and padding
def collate_fn(batch):
    img_paths, captions_ = zip(*batch)
    images = [Image.open(p).convert("RGB") for p in img_paths]
    image_inputs = [image_processor(img, return_tensors="pt") for img in images]
    merged_image_inputs = {}
    for k in image_inputs[0]:
        merged_image_inputs[k] = torch.cat([d[k] for d in image_inputs], dim=0)
    input_ids = [qwen_tokenizer(c, return_tensors="pt", truncation=True, max_length=32).input_ids[0] for c in captions_]
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=qwen_tokenizer.pad_token_id)
    return merged_image_inputs, input_ids_padded

# DataLoader
dataloader = DataLoader(pairs, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for image_inputs_batch, input_ids_batch in tqdm(dataloader, desc=f"Training epoch {epoch+1}"):
        for k in image_inputs_batch:
            image_inputs_batch[k] = image_inputs_batch[k].to(device)
        input_ids_batch = input_ids_batch.to(device)

        # Skip batches with only padding
        if (input_ids_batch != qwen_tokenizer.pad_token_id).sum().item() == 0:
            continue

        image_features = vision_encoder(**image_inputs_batch).pooler_output
        image_features_proj = clip_model.visual_projection(image_features)
        image_latent = projection(image_features_proj.float())

        text_embeds = qwen_model.model.embed_tokens(input_ids_batch)
        decoder_input = torch.cat([image_latent.unsqueeze(1), text_embeds], dim=1)

        # Pad labels for image token alignment
        labels = input_ids_batch.clone()
        labels = torch.cat([
            torch.full((labels.shape[0], 1), -100, dtype=labels.dtype, device=labels.device),
            labels
        ], dim=1)
        outputs = qwen_model(inputs_embeds=decoder_input, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(projection.parameters(), max_norm=1.0)
        optimizer.step()

        tqdm.write(f"Loss: {loss.item():.4f}")

print("Training loop complete.")

Epoch 1/3


Training epoch 1:   0%|          | 1/1012 [00:02<37:43,  2.24s/it]

Loss: 8.8220


Training epoch 1:   0%|          | 2/1012 [00:03<26:48,  1.59s/it]

Loss: nan


...

Loss: nan


Training epoch 1:   7%|▋         | 71/1012 [00:49<11:38,  1.35it/s]

Loss: nan


Training epoch 1:   7%|▋         | 72/1012 [00:50<11:20,  1.38it/s]

Loss: nan
